# CPA Mappings

Produced by Elias Kellow

## 1.0 Introduction

So the work you did with the CPA classification 

This work was undertaken as the current Producer prices system has products based on CPA 2008,  this means that they are 1 or 2 digit levels e.g 01 or 20.5,   The new Producer Prices Chain Linked system is using the National Accounts accepted standard CPA product codes,  i.e CPA_A01 or CPA_C205.   However this classification was also provided with higher product level to 6 digit products and therefore we needed a way to apply the correct Child code to its parent code for all letter industries A to U, using the supplied level spreadsheet from prices division . 


## 2.0 Synopsis

Read the file 'raw_data_cpa_code.xlsx' that has the CPA codes and 'CPA descriptions.' Read the file 'raw_data_hierarchies.xlsx' which has the hierarchical levels, and **'short codes'** are derived from this file. Both files have been processed and this allows them to be joined using short codes (how short codes are derived is decribed below). Thus, CPA codes and descriptions are linked with hierarchical levels, and they are linked with CPA parent codes. In the file 'raw_data_cpa_code.xlsx' strings have been split on ':' where everything before is the CPA code and everyhting after is the 'CPA description.' The CPA code has been split further to get a short code which is used to link with the other file called 'raw_data_hierarchies.xlsx.' Also, the CPA parent code has been associated with the relevant CPA code. Thus, the final table relates the description, CPA code, short code and hierarchies. 

## 3.0 Processing

### 3.1 Aim: Read 'raw_data_cpa_code.xlsx' and get 'cp code,' 'cp description' and derive 'short code.'

Goal: Select specific rows of the file raw_data_cpa_code.xlsx, get the **cp code**, **cp description** **short code**(after the first four digits of the cp code).


Here's the code to read the file 'raw_data_cpa_code.xlsx,' and put into a dataframe called **df_products**; then select rows 4 to 5528 and the last 4 rows of **df_products**; place the selected rows into two seperate dataframes; append these dataframes together.  In this appended dataframe, split the column into two columns using ':', where the new column called 'cp.code' is the **CP code** and 'cp.desc' is the **CP Description**.  Then, split the column 'cp.code' to get the **short code**.

In [1]:
#aim is to get a list of codes
#step 1 - import pandas
import pandas as pd

#step 2 - read excel file from sheet1, and create names 'level' and 'code,' with no header
data_sht1 = pd.read_excel(r'raw_data_cpa_code.xlsx', 'Sheet1', header=None, names=['level', 'code'])
#step 3 - put the data into a dataframe with column named PROD
df_products = pd.DataFrame(data_sht1, columns=['code'])

#step 4 - select rows 4 to 5528 of the dataframe
df_products_begin = df_products[3:5527]

#step 5 - select the last four rows of the dataframe
df_products_end = df_products[5554:]

#step 6 - append the dataframes together
df_products_total = df_products_begin.append(df_products_end)

#step 7 - split the column 'code' into two using the delimeter ':'
df_products_total['cp.code'], df_products_total['cp.desc'] = df_products_total['code'].str.split(':', 1).str

#aim is to split the cp code to get the short code form
#step 8 - create a new column cp.code.2 so that it can be slit to get the 'short.code'

df_products_total['cp.code.2']=df_products_total['cp.code']


#step 7 - split the column 'code' into two using the delimeter ':'
df_products_total['cpa'], df_products_total['short.code'] = df_products_total['cp.code.2'].str.split('_', 1).str

#step 8 - insert 'TOTAL' into the column 'short.code' for the index row 5554
df_products_total.loc[5554, 'short.code'] = 'TOTAL'


#step 9 - remove duplicates of the column
df_products_total.drop_duplicates('short.code', keep='last', inplace=True)

#step 10 - remove trailing and leading white spaces for all rows of the dataframe df_products_total

df_products_total.columns = df_products_total.columns.str.strip()
df_products_total.tail()
#print(len(df_products_total))

,code,cp.code,cp.desc,cp.code.2,cpa,short.code
5554,TOTAL: TOTAL,TOTAL,TOTAL,TOTAL,TOTAL,TOTAL
5555,"CPA_A: Products of agriculture, forestry a...",CPA_A,"Products of agriculture, forestry and fishing",CPA_A,CPA,A
5556,"CPA_A01: Products of agriculture, hunt...",CPA_A01,"Products of agriculture, hunting and related ...",CPA_A01,CPA,A01
5557,CPA_B: Mining and quarrying,CPA_B,Mining and quarrying,CPA_B,CPA,B
5558,CPA_C: Manufactured products,CPA_C,Manufactured products,CPA_C,CPA,C


### 3.2 Aim: Read the file 'raw_data_hierarchies.xlsx and process the file so that It can be joined with data from the file 'raw_data_cpa_code.xlsx'

Here is the code to read the file **'raw_data_hierarchies.xlsx**,' **remove duplicates** of the column 'codes.' Here's Remove white leading and tailing spaces in order to do a join later.

In [2]:
#step 10 - read excel file - raw_data_hierarchies - from the sheet called 'CPA 2.1 - CPA 2008_20191011_152', 
#where the first row is the header

data_sht2 = pd.read_excel(r'raw_data_hierarchies.xlsx', 'CPA 2.1 - CPA 2008_20191011_152',header=0)
#step 11 - put the csv file into a dataframe, and rename columns
df_parents = pd.DataFrame(data_sht2).rename(columns ={'CPA2.1':'codes', 'CPA2.1-LEVEL':'hierarchy'})

#step 13 - remove duplicate row based on the column codes
df_parents.drop_duplicates(subset='codes', keep="last", inplace=True)

#step 14 - remove trailing and leading white spaces

df_parents.columns = df_parents.columns.str.strip()

df_parents.head()
#print(len(df_parents))

,codes,CPA2008,hierarchy,CPA2008-LEVEL,CPA2.1-4D,CPA2008-4D,DIFFERENT?
0,A,A,1,1,A,A,SAME
1,01,01,2,2,01,01,SAME
2,01.1,01.1,3,3,01.1,01.1,SAME
3,01.11,01.11,4,4,01.11,01.11,SAME
4,01.11.1,01.11.1,5,5,01.11,01.11,SAME


Here the code removes '.' for each row in the column called 'codes,'  and concatenates/places in front each code in the list code_2 with its parent.  For example, 'A' and '01' are concatenated together to produce 'A01'. Another example is 'A' and '011' are concatenated to produce 'A011'.

In [3]:
#step 14 - remove '.' from the column codes
df_parents['codes']=df_parents['codes'].replace({'\.':''}, regex=True)


#step 15 - create a function where -concerning the column codes in the file 'raw_data_hierarchies.xlsx' - if the 
#length of a value of row one, then paste that value into the column 'temp.parent,' otherwise paste None.

def parent_row (row):
    if len(row['codes'])==1:
        return row['codes']
    if len(row['codes'])>1:
        return None

#step 16 - apply the function 'parent_row' for each row, then remove

df_parents['temp.parent']=df_parents.apply(lambda row:parent_row(row), axis=1).fillna(method='ffill')


#step 17 - create a function where - concerning the column codes in the dataframe df_parents - if the 
#length of a value of row one, then paste that value into the column 'short.code,', otherwise concatenate the respective 
#value of the column 'temp.parent' and 'codes'. e.g. A and 01 are concenated to produce A01, because the length
# of 01 is greater than one.
def concat_row (row):
    if len(row['codes'])==1:
        return row['codes']
    if len(row['codes'])>1:
        return row['temp.parent']+row['codes']

#step 16 - apply the function 'parent_row' for each row, then remove

df_parents['short.code']=df_parents.apply(lambda row:concat_row(row), axis=1)

#step 17 - create a function
df_parents.head()

,codes,CPA2008,hierarchy,CPA2008-LEVEL,CPA2.1-4D,CPA2008-4D,DIFFERENT?,temp.parent,short.code
0,A,A,1,1,A,A,SAME,A,A
1,01,01,2,2,01,01,SAME,A,A01
2,011,01.1,3,3,01.1,01.1,SAME,A,A011
3,0111,01.11,4,4,01.11,01.11,SAME,A,A0111
4,01111,01.11.1,5,5,01.11,01.11,SAME,A,A01111


### 3.3 Aim: Link hierarchies with CPA codes

Here's the code that links heirarchical levels and CPA codes by joining the data from the file 'raw_data_cpa_code.xlsx' and 'raw_data_hierarchies.xlsx', i.e. join the dataframe df_parents and df_products_total using the column 'short.code.'  Also, find the parent CPA code using a function parent_code.

In [4]:
#step 17 - remove trailing and leading white spaces for all rows in the dataframe df_parents

df_parents.columns = df_parents.columns.str.strip()

#step 18 - join the dataframe df_products_total and df_parents using the column 'short.code'
df_joined = df_products_total.merge(df_parents, on = 'short.code', how = 'inner')
#print(len(df_products_total))
#print(len(df_parents))
#print(len(df_joined))

df_joined.columns = df_joined.columns.str.strip()

#step 17 - create a function finds the parent cpa code
def cpa_parent_row (row):
    if len(row['short.code'])==1:
        
        return 'TOTAL'
    
    if len(row['short.code'])==3:
        
        return row['cp.code'][:-2]
    
    if len(row['short.code'])>3:
        
        return row['cp.code'][:-1]

#step 16 - apply the function 'parent_row' for each row, then remove

df_joined['parent.cpa']=df_joined.apply(lambda row:cpa_parent_row(row), axis=1)

df_cpa_map=df_joined[['cp.code', 'cp.desc', 'hierarchy', 'parent.cpa', 'short.code']]
df_cpa_map.tail()

df_cpa_map.to_excel('hierarchy5.xlsx', index=None)
#print(len('CPA_A'))

# 4.0 Data Quality Analysis

Here's the code to see if they are cp codes that exist 'raw_data_cpa_code.xlsx.' The current output is **hierarchy5.xlsx** (see the previous code line above).  All previous previous cells needs to be run for the below code to work.  Also, **hierarchy5.xlsx** and this jupyter notebook has to be in the same working directory.

In [50]:
#step a - read the file 'hierarchy4.xlsx
data_sht3 = pd.read_excel(r'hierarchy5.xlsx', 'Sheet1',header=0)
#step b - put the excel object into a dataframe and rename a column
df_test = pd.DataFrame(data_sht3).rename(columns ={'desc':'cp.code'})


#step c - remove trailing and leading white spaces

df_test.columns = df_test.columns.str.strip()



df_merge=df_test.merge(df_products_total, on = 'cp.code', how = 'right')

df_not_join = df_merge[df_merge['codes'].isnull()]
#df_merge.tail()
#df_test.tail()

#print(len(df_merge[df_merge['codes'].isnull()]))
df_not_join.to_excel('notjoin.xlsx', index=None)

# 5.0 Environment

Running under: Windows 7 x64 (build 7601) Service Pack 1

I was using Jupyter notebook.  The version of the notebook server is 5.0.0 and is running on:

- Python 3.6.1 |Anaconda 4.4.0 (64-bit)| (default, May 11 2017, 13:25:24) [MSC v.1900 64 bit (AMD64)]
    


In [37]:
print(len(df_products_total['code'].tolist()))

5529
